## Homework 3: Classification

1) Load in the dataset https://d1pqsl2386xqi9.cloudfront.net/notebooks/Default.csv into a pandas dataframe

2) Convert default and student to categorical variables.

3) Make boxplots of all of the variables and a scatter plot of of (balance,income) with the color of the outcome of default labeled red if yes and blue if no.

4) Train a model to predict if a student defaults on their loan or not using LogisticRegression based soley on the balance and income variables. Plot the decision boundary in the scatter plot you made in 3) and compute the AUC ROC.

5) Repeat 4) but with the student variable included. Optimize using Ridge regression and plot the ranking of coefficients. How does performance improve with the added variable?

6) Using cross validation, check that the model you built in 5) is robust (generalizes to other random folds of the dataset).

7) Train a Decision Tree classifier with maximum depth 5 and plot the decision tree. How does performance compare?

8) Find the optimal decision tree depth and compute the AUC ROC.

In [3]:
import statsmodels.api as sm
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
import pylab
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import re

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import pylab as plt
import seaborn
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import numpy.random as nprnd
import random
import json

pd.set_option('display.max_columns', 500)
df = pd.read_csv('https://d1pqsl2386xqi9.cloudfront.net/notebooks/Default.csv ')
df.head()

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700
2,3,No,No,1073.549164,31767.138947
3,4,No,No,529.250605,35704.493935
4,5,No,No,785.655883,38463.495879


In [13]:
df_default = pd.get_dummies(df['default'])
df_student = pd.get_dummies(df['student'])
df_final = pd.concat([df[['Unnamed: 0', 'balance', 'income']],df_default, df_student],axis=1)
df_final.head()

,Unnamed: 0,balance,income,No,Yes,No,Yes
0,1,729.526495,44361.625074,1.0,0.0,1.0,0.0
1,2,817.180407,12106.134700,1.0,0.0,0.0,1.0
2,3,1073.549164,31767.138947,1.0,0.0,1.0,0.0
3,4,529.250605,35704.493935,1.0,0.0,1.0,0.0
4,5,785.655883,38463.495879,1.0,0.0,1.0,0.0
